# Sesi 6 – Saluran Berbilang Langkah (Rancang → Laksana → Perhalusi)

Menggunakan fungsi penghalaan untuk memilih model bagi setiap langkah, kemudian memperhalusi jawapan akhir.

## Objektif

- Memahami bagaimana untuk membahagikan tugas kompleks kepada langkah-langkah yang lebih kecil.
- Menggunakan model yang berbeza untuk setiap langkah dalam saluran.
- Memperhalusi jawapan akhir berdasarkan hasil daripada langkah sebelumnya.

## Gambaran Keseluruhan

Pendekatan ini melibatkan tiga langkah utama:

1. **Rancang**: Kenal pasti langkah-langkah yang diperlukan untuk menyelesaikan tugas.
2. **Laksana**: Gunakan model yang sesuai untuk melaksanakan setiap langkah.
3. **Perhalusi**: Gabungkan hasil daripada langkah-langkah sebelumnya untuk menghasilkan jawapan akhir yang lebih baik.

## Contoh

### Senario: Menulis Artikel

Katakan anda ingin menulis artikel tentang topik tertentu. Anda boleh membahagikan tugas ini kepada beberapa langkah:

1. **Rancang**: Tentukan struktur artikel dan subtopik utama.
2. **Laksana**: Tulis kandungan untuk setiap subtopik.
3. **Perhalusi**: Gabungkan semua bahagian dan perbaiki artikel untuk memastikan ia mengalir dengan baik.

### Kod Contoh

```python
# Langkah 1: Rancang
plan = planning_model("Apakah langkah-langkah untuk menulis artikel tentang perubahan iklim?")

# Langkah 2: Laksana
content = []
for step in plan:
    result = execution_model(f"Tulis kandungan untuk: {step}")
    content.append(result)

# Langkah 3: Perhalusi
final_article = refinement_model("Gabungkan dan perhalusi kandungan berikut: " + " ".join(content))
```

## Kelebihan

- **Modulariti**: Setiap langkah boleh dioptimumkan secara berasingan.
- **Fleksibiliti**: Model yang berbeza boleh digunakan untuk tugas yang berbeza.
- **Kualiti**: Memperhalusi jawapan akhir meningkatkan ketepatan dan koherensi.

## Cabaran

- **Pengurusan Kompleksiti**: Memerlukan perancangan yang teliti untuk memastikan setiap langkah diselaraskan.
- **Kos Pengiraan**: Menggunakan model berbeza untuk setiap langkah boleh menjadi mahal.

[!TIP] Gunakan fungsi penghalaan untuk memilih model yang paling sesuai berdasarkan keperluan setiap langkah.

## Kesimpulan

Pendekatan saluran berbilang langkah membolehkan anda menangani tugas kompleks dengan lebih sistematik. Dengan membahagikan tugas kepada langkah-langkah yang lebih kecil, anda boleh meningkatkan kecekapan dan kualiti hasil akhir.


### Penjelasan: Pemasangan Kebergantungan
Memasang `foundry-local-sdk` dan `openai` yang diperlukan untuk panggilan chat setiap langkah. Selamat untuk dijalankan semula.


# Senario
Saluran model berbilang langkah yang: (1) Merancang tugas kepada langkah-langkah yang berasingan, (2) Melaksanakan setiap langkah dengan pemilihan model berdasarkan niat, (3) Menyempurnakan jawapan akhir yang disintesis. Menunjukkan penggabungan keupayaan SLM yang pelbagai untuk kecekapan sambil mengekalkan kualiti.


In [13]:
!pip install -q foundry-local-sdk openai

### Penjelasan: Import Teras
Mengimport regex untuk pengesanan niat, Foundry Local manager untuk lampiran per-alias, dan OpenAI client untuk penyelesaian sembang.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Penjelasan: Katalog Keupayaan & Peraturan
Menentukan katalog yang menyedari keupayaan dan peraturan regex yang digunakan untuk memetakan teks langkah kepada kategori niat (kod, ringkasan, klasifikasi, umum). Nilai keutamaan yang lebih rendah akan menang sekiranya terdapat persaingan apabila pelbagai model menyokong satu niat.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### Penjelasan: Niat, Pemilihan Model & Pembantu Chat
Menyediakan:
- `detect_intent` untuk klasifikasi berdasarkan regex.
- `pick_model` untuk memilih alias terbaik berdasarkan keupayaan + keutamaan.
- `chat` pembungkus kemudahan yang memulakan klien bagi setiap alias dan mengembalikan respons satu pusingan.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### Penjelasan: Fungsi Saluran Berbilang Langkah
Melaksanakan saluran: rancangan → langkah-langkah analisis → laksanakan setiap satu dengan penghalaan berdasarkan niat → perhalusi output gabungan. Mengembalikan kamus berstruktur untuk pemeriksaan atau penilaian.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### Penjelasan: Jalankan Contoh Tugas
Melaksanakan keseluruhan aliran kerja pada tugas yang berorientasikan penstrukturan semula yang menggambarkan niat bercampur (kod + ringkasan). Kamus hasil menunjukkan rancangan mentah, output setiap langkah dengan alias model yang dipilih, dan jawapan akhir yang disintesis.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### Penjelasan: Paparkan Objek Hasil
Menunjukkan output saluran paip yang berstruktur untuk pemeriksaan pantas atau penilaian seterusnya (contohnya, mengukur kualiti langkah atau keberkesanan penambahbaikan).



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
